
Open the notebook in Colab

<a href="https://colab.research.google.com/github/gbarbalinardo/ballistico-examples/blob/master/crystal_presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Necessary Repos


In [0]:
!pip install git+https://<USERNAME>:<PASSWORD>@github.com/gbarbalinardo/ballistico.git

# Remote Fetch Necessary Files


In [0]:
import os
if not os.path.exists('forcefields.zip'):
  !wget http://chemac196.ucdavis.edu/data-html/forcefields.zip
  !unzip forcefields.zip
  !rm -r forcefields.zip

if not os.path.exists('nanowire.xyz'):
  !wget http://chemac196.ucdavis.edu/data-html/nanowire.xyz

# Remote Fetch Precomplied LAMMPS


In [0]:
!wget http://drt.io/dl/judqh/lmp-tesla-t4-intel-xeon.gz
!tar xvzf lmp-tesla-t4-intel-xeon.gz
!rm /content/lmp-tesla-t4-intel-xeon.gz
!ln -s /content/lammps/src/lmp_gpu /usr/local/bin/lmp_gpu

# Navigate back to lammps source foder when for 
# later LAMMPS-Python integration

%cd /content/lammps/src

print('\n')
print('Remote Fetching precomplied LAMMPS is finished!')

#Integrate LAMMPS Into Python 

In [0]:
!make install-python

# Copy executable file to where the python Module locates

import shutil 
src_path = '/usr/lib/python3/dist-packages/liblammps.so'
dist_path = '/usr/local/lib/python3.6/dist-packages/liblammps.so'
shutil.copyfile(src_path,dist_path)

# Naviagate back to main folder before simulation

%cd /content

print('\n')
print('LAMMPS-Python Integration is completed!')

# Thermal Transport Simulation for Silicon-Bulk

## Import modules and create finite diffence object

In [0]:
from ase.build import bulk
from ase.calculators.lammpslib import LAMMPSlib
from ballistico.io_helper import *
import ballistico.plotter as plotter
from ballistico import conductivity
from ballistico.finitedifference import FiniteDifference
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')


# We start from the atoms object

atoms = bulk('Si', 'diamond', a=5.432)


supercell = np.array([3, 3, 3])
lammps_inputs = {
      'lmpcmds': [
          'pair_style tersoff',
          'pair_coeff * * forcefields/Si.tersoff Si'],

      'log_file': 'lammps-si-bulk.txt',
      'keep_alive':True}

# Create a finite difference object

finite_difference = FiniteDifference(atoms=atoms,
                    supercell=supercell,
                    calculator=LAMMPSlib,
                    calculator_inputs=lammps_inputs,
                    is_reduced_second=False,
                    folder='si-bulk-fd')


## Create phonons object


In [0]:
k = 7
kpts = [k, k, k]
temperature = 300
is_classic = False
k_label = str(int(np.prod(kpts)))

# Create a phonon object

phonons = Phonons(finite_difference=finite_difference,
                kpts=kpts,
                is_classic=is_classic,
                temperature=temperature,
                folder='si-bulk-ald-' + k_label)

## Calculate conductivities for infinite-size sample


In [0]:
# Calculate conductivity  with direct inversion approach (inverse)

print('Inverse conductivity matrix: ')
print(phonons.conductivity(method='inverse').sum(axis=0))
print('Inverse conductivity [W/(m•K)]: %.3f '
  %np.mean(np.diag(phonons.conductivity(method='inverse').sum(axis=0))))
print('\n')

# Calculate conductivity  with  relaxation time approximation (rta)

print('Rta conductivity matrix: ')
print(phonons.conductivity(method='rta').sum(axis=0))
print('Rta conductivity [W/(m•K)]: %.3f '
%np.mean(np.diag(phonons.conductivity(method='rta').sum(axis=0))))
print('\n')

# Calculate conductivity  with  self-consistent approach (sc)

max_n_iters=11
sc_cond= phonons.conductivity(method='sc', max_n_iterations=max_n_iters)

print('Self-consistent conductivity matrix: ')
print(sc_cond[1].sum(axis=0))
print('Self Consistent conductivity [W/(m•K)]: %.3f '%
      np.mean(np.diag(sc_cond[0].sum(axis=0))))
print('\n')

## Visualize Basic Quantities from the simulation


In [0]:
# Plot dispersion relation and group velocity in each direction

freq_full = phonons.frequencies.flatten()
plotter.plot_dispersion(phonons,n_k_points=int(k_label))
print('\n')

# Plot heat compacity vs frequency

print('\n')
plt.figure()
plt.scatter(freq_full,1e23*phonons.c_v,s=15)
plt.ylabel (r"$C_{v}$ [$10^{23}$ J/K]", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(0.9*1e23*phonons.c_v[phonons.c_v>0].min(), 
         1.05*1e23*phonons.c_v.max())
plt.show()

# Plot phase space vs frequency

print('\n')
plt.figure()
plt.scatter(freq_full,phonons.ps,s=15)
plt.ylabel ("Phase space", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(phonons.ps.min(), phonons.ps.max())
plt.show()

## Calculate and Visualize Advanced Properties

In [0]:
# Advanced properties can also be 
# calculated by "lazy-loading" properties
# calculated during the simulation.

tau_rta = phonons.gamma[phonons.gamma!=0]**(-1)
freq = freq_full[phonons.gamma!=0]
velocities = phonons.velocities

velocity_norms_x = np.sqrt((velocities[:, :, 0] ** 2)).flatten(order='C')
velocity_norms_y = np.sqrt((velocities[:, :, 1] ** 2)).flatten(order='C')
velocity_norms_z = np.sqrt((velocities[:, :, 2] ** 2)).flatten(order='C')
velocity_norms = (velocity_norms_x + velocity_norms_y + velocity_norms_z)/3

mfp_rta = np.multiply(velocity_norms[phonons.gamma != 0], tau_rta)

# Plot phonon life time under rta approach vs frequency

print('\n')
plt.figure()
plt.scatter(freq,tau_rta,s=15)
plt.ylabel (r"$\tau_{rta}$ [ps]", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(0.95*tau_rta.min(), 1.05*tau_rta.max())
plt.yscale('symlog')
plt.show()

# Plot mean free path (mfp) under rta approach vs frequency

print('\n')
plt.figure()
plt.scatter(freq,0.1*mfp_rta,s=15)
plt.ylabel (r"$\lambda_{rta}$ [$\AA$]", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(-0.5, 0.1*1.05*mfp_rta.max())
plt.yscale('symlog')
plt.show()

## Visualize $\kappa_{cum}$ Vs. $\nu$ , $\kappa_{sc}$ Vs. # of iterations  and finite-size simulation




In [0]:
# Conductivity can also be illustrated in a cumulative manner

freq_argsort_index = np.argsort(freq_full)

rta_cond_per_mode = phonons.conductivity(method='rta')
rta_cond_per_mode_flattened_x = rta_cond_per_mode[:, 0, 0].flatten(order='C')
rta_cond_per_mode_flattened_y = rta_cond_per_mode[:, 1, 1].flatten(order='C')
rta_cond_per_mode_flattened_z = rta_cond_per_mode[:, 2, 2].flatten(order='C')
rta_cond_per_mode_flattened = (rta_cond_per_mode_flattened_x
                                         + rta_cond_per_mode_flattened_y +
                                         rta_cond_per_mode_flattened_z)/3

rta_cond_per_mode_argsort =  rta_cond_per_mode_flattened[freq_argsort_index]
kappa_cum_rta_by_freq = np.cumsum(rta_cond_per_mode_argsort)

inverse_cond_per_mode = phonons.conductivity(method='inverse')
inverse_cond_per_mode_flattened_x = inverse_cond_per_mode[:, 0, 0].flatten(order='C')
inverse_cond_per_mode_flattened_y = inverse_cond_per_mode[:, 1, 1].flatten(order='C')
inverse_cond_per_mode_flattened_z = inverse_cond_per_mode[:, 2, 2].flatten(order='C')
inverse_cond_per_mode_flattened = (inverse_cond_per_mode_flattened_x
                                         + inverse_cond_per_mode_flattened_y +
                                         inverse_cond_per_mode_flattened_z)/3

inverse_cond_per_mode_argsort =  inverse_cond_per_mode_flattened[freq_argsort_index]
kappa_cum_inverse_by_freq = np.cumsum(inverse_cond_per_mode_argsort)


sc_cond_per_mode = phonons.conductivity(method='sc',max_n_iterations=max_n_iters)[0]
sc_cond_per_mode_flattened_x = sc_cond_per_mode[:, 0, 0].flatten(order='C')
sc_cond_per_mode_flattened_y = sc_cond_per_mode[:, 1, 1].flatten(order='C')
sc_cond_per_mode_flattened_z = sc_cond_per_mode[:, 2, 2].flatten(order='C')
sc_cond_per_mode_flattened = (sc_cond_per_mode_flattened_x
                                         + sc_cond_per_mode_flattened_y +
                                         sc_cond_per_mode_flattened_z)/3

sc_cond_per_mode_argsort =  sc_cond_per_mode_flattened[freq_argsort_index]
kappa_cum_sc_by_freq = np.cumsum(sc_cond_per_mode_argsort)

# Plot self-consistent conductivity vs # of iterations

sc_cond = phonons.conductivity(method='sc', max_n_iterations=10)[1]
sc_conductivity = (sc_cond[:,0,0] + sc_cond[:,1,1] + sc_cond[:,2,2])/3

print('\n')
plt.figure()
plt.plot(sc_conductivity ,label ="$\kappa_{sc} & ")
plt.ylabel (r"$\kappa$ [W/(m•K)]", fontsize=25, fontweight='bold')
plt.xlabel ("# of iterations", fontsize=25, fontweight='bold')
plt.grid()
plt.show()

# Plot cumulative condutivity (kappa) vs frequency

print('\n')
plt.figure()
plt.plot(freq_full[freq_argsort_index], 
         kappa_cum_rta_by_freq, 'r-',label='$\kappa_{rta}$')

plt.plot(freq_full[freq_argsort_index], kappa_cum_inverse_by_freq, 
         'k-',label='$\kappa_{inverse}$')

plt.plot(freq_full[freq_argsort_index], 
         kappa_cum_sc_by_freq,'bs',markersize=5,label='$\kappa_{sc}$')


plt.xlabel("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylabel(r'Cumulative $\kappa$ [W/(m•K)]', fontsize=25, fontweight='bold')
plt.grid()
plt.legend(loc=4, prop={'size': 15})
plt.show()

# Peform simulation for finite-size sample

cond_vs_len_rta = []
cond_vs_len_sc = []

lengths = np.outer(np.array([10, 100, 1000, 10000, 100000]), 
              np.array([1, 2, 5])).flatten(order='C')
for length in lengths:
  cond_rta = phonons.conductivity(method='rta', length=length, 
                        axis=0).sum(axis=0)
  cond_vs_len_rta.append(cond_rta[0, 0])
  cond_sc = phonons.conductivity(method='sc', max_n_iterations = 10,
                                 length=length, axis=0)[0].sum(axis=0)
  cond_vs_len_sc.append(cond_sc[0, 0])

print('\n')
plt.figure()
plt.plot(lengths, np.array(cond_vs_len_rta), '-b', label='rta')
plt.plot(lengths, np.array(cond_vs_len_sc), '-', label='sc')
plt.legend(loc='best')
plt.xlabel('$\ell$ [nm]',  fontsize=25, fontweight='bold')
plt.ylabel(r"$\kappa$ [W/(m•K)]",  fontsize=25, fontweight='bold')
plt.grid()
plt.xscale('log')
plt.show()

# Thermal Transport Simulation for Silicon-Nanowire (si-nw)



## Import modules and create finite diffence object

In [0]:
import ase
from ase.build import bulk
from ase.calculators.lammpslib import LAMMPSlib
from ballistico.io_helper import *
import ballistico.plotter as plotter
from ballistico.finitedifference import FiniteDifference
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')


# We start from the atoms object

atoms = ase.io.read('nanowire.xyz')


# Duplicate supercell only in z direction

supercell = np.array([1, 1, 3])
lammps_inputs = {
      'lmpcmds': [
          'pair_style tersoff',
          'pair_coeff * * forcefields/Si.tersoff Si'],

      'log_file': 'lammps-si-nw.txt',
      'keep_alive':True}

# Create a finite difference object

finite_difference = FiniteDifference(atoms=atoms,supercell=supercell,
                    calculator=LAMMPSlib,
                    calculator_inputs=lammps_inputs,
                    is_reduced_second=False,
                    folder='si-nw-fd')

## Create phonons object


In [0]:
k = 7
is_classic = False
kpts = [1, 1, k]
temperature = 300
k_label = str(int(np.prod(kpts)))

# Create a phonon object

phonons = Phonons(finite_difference=finite_difference,
                kpts=kpts,
                is_classic=is_classic,
                temperature=temperature,
                folder='si-nw-ald-' + k_label)

## Calculate conductivities for infinite-size sample


In [0]:
# Calculate conductivity for silicon nanowire for infinity samples
# Define volume ratio 

volume_ratio = 3.41

# Calculate conductivity  with direct inversion approach (inverse)

print('Inverse conductivity matrix: ')
print(volume_ratio*phonons.conductivity(method='inverse').sum(axis=0))
inverse_cond = phonons.conductivity(method='inverse').sum(axis=0)[2,2]
print('Inverse conductivity [W/(m•K)]: %.3f'%(volume_ratio*inverse_cond))
print('\n')

# Calculate conductivity with relaxation time approximation approach (rta)

print('Rta conductivity matrix: ')
print(volume_ratio*phonons.conductivity(method='rta').sum(axis=0))
rta_cond = phonons.conductivity(method='rta').sum(axis=0)[2,2]
print('Rta conductivity [W/(m•K)]: %.3f '%(volume_ratio*rta_cond))
print('\n')

max_n_iters=57
sc_cond= phonons.conductivity(method='sc', max_n_iterations=max_n_iters)

# Calculate conductivity with self-consistent approach (sc)

print('Self-consistent conductivity matrix: ')
print(sc_cond[1].sum(axis=0))
sc_cond_z = sc_cond[0].sum(axis=0)[2,2]
print('Self Consistent conductivity [W/(m•K)]: %.3f '%(volume_ratio*sc_cond_z))

## Visualize Basic Quantities from the simulation


In [0]:
# Plot dispersion relation and group velocity in z-direction

plotter.plot_dispersion(phonons,symmetry='nw', n_k_points=400)
print('\n')

# Plot heat compacity vs frequency

freq_full = phonons.frequencies.flatten()

print('\n')
plt.figure()
plt.scatter(freq_full,1e23*phonons.c_v,s=15)
plt.ylabel (r"$C_{v}$ [$10^{23}$ J/K]", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(0.9*1e23*phonons.c_v[phonons.c_v>0].min(), 
          1.05*1e23*phonons.c_v.max())
plt.show()

# Plot phase space vs frequency

print('\n')
plt.figure()
plt.scatter(freq_full,phonons.ps,s=15)
plt.ylabel ("Phase space", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(phonons.ps.min(), phonons.ps.max())
plt.show()

## Calculate and Visualize Advanced Properties

In [0]:
# Advanced properties can also be 
# calculated by "lazy-loading" properties
# calculated during the simulation.

tau_rta = phonons.gamma[phonons.gamma!=0]**(-1)
freq = freq_full[phonons.gamma!=0]
velocities = phonons.velocities
velocity_norms = np.sqrt((velocities[:, :, 2] ** 2)).flatten(order='C')
mfp_rta = np.multiply(velocity_norms[phonons.gamma != 0], tau_rta)

# Plot phonon life time under rta approach vs frequency

print('\n')
plt.figure()
plt.scatter(freq,tau_rta,s=15)
plt.ylabel (r"$\tau_{rta}$ [ps]", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(tau_rta.min(), tau_rta.max())
plt.yscale('symlog')
plt.show()

# Plot mean free path (mfp) under rta approach vs frequency

print('\n')
plt.figure()
plt.scatter(freq,0.1*mfp_rta,s=15)
plt.ylabel (r"$\lambda_{rta}$ [$\AA$]", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylim(-0.5, 0.1*1.05*mfp_rta.max())
plt.yscale('symlog')
plt.show()

## Visualize $\kappa_{cum}$ Vs. $\nu$ , $\kappa_{sc}$ Vs. # of iterations  and finite-size simulation




In [0]:
# Plot self-consistent conductivity vs # of iterations

sc_cond = phonons.conductivity(method='sc', max_n_iterations=57)[1]
sc_conductivity = sc_cond[:,2,2]

print('\n')
plt.figure()
plt.plot(volume_ratio*sc_conductivity ,label ="$\kappa_{sc} & ")
plt.ylabel (r"$\kappa$ [W/(m•K)]", fontsize=25, fontweight='bold')
plt.xlabel ("# of iterations", fontsize=25, fontweight='bold')
plt.grid()
plt.show()

# Conductivity can also be illustrated in a cumulative manner

freq_argsort_index = np.argsort(freq_full)

rta_cond_per_mode = phonons.conductivity(method='rta')
rta_cond_per_mode_flattened = rta_cond_per_mode[:, 2, 2].flatten(order='C')

rta_cond_per_mode_argsort =  rta_cond_per_mode_flattened[freq_argsort_index]
kappa_cum_rta_by_freq = np.cumsum(rta_cond_per_mode_argsort)


inverse_cond_per_mode = phonons.conductivity(method='inverse')
inverse_cond_per_mode_flattened = inverse_cond_per_mode[:, 2, 2].flatten(order='C')

inverse_cond_per_mode_argsort =  inverse_cond_per_mode_flattened[freq_argsort_index]
kappa_cum_inverse_by_freq = np.cumsum(inverse_cond_per_mode_argsort)

sc_cond_per_mode = phonons.conductivity(method='sc',max_n_iterations=max_n_iters)[0]
sc_cond_per_mode_flattened = sc_cond_per_mode[:, 2, 2].flatten(order='C')

sc_cond_per_mode_argsort =  sc_cond_per_mode_flattened[freq_argsort_index]
kappa_cum_sc_by_freq = np.cumsum(sc_cond_per_mode_argsort)


# Plot cumulative condutivity (kappa) vs frequency

print('\n')
plt.figure()
plt.plot(freq_full[freq_argsort_index], 
        volume_ratio*kappa_cum_rta_by_freq, 'r-',label='$\kappa_{rta}$')


plt.plot(freq_full[freq_argsort_index],  volume_ratio*kappa_cum_inverse_by_freq, 
          'k-',label='$\kappa_{inverse}$')

plt.plot(freq_full[freq_argsort_index], 
          volume_ratio *kappa_cum_sc_by_freq,'bs',
          markersize=5,label='$\kappa_{sc}$')


plt.xlabel("$\\nu$ [Thz]", fontsize=25, fontweight='bold')
plt.ylabel(r'Cumulative $\kappa$ [W/(m•K)]', fontsize=25, fontweight='bold')
plt.grid()
plt.legend(loc=4, prop={'size': 15})
plt.show()

# Peform simulation for finite-size sample

cond_vs_len_rta = []
cond_vs_len_sc = []

lengths = np.outer(np.array([10, 100, 1000, 10000, 100000]), 
              np.array([1, 2, 5])).flatten(order='C')
for length in lengths:
  cond_rta = phonons.conductivity(method='rta', length=length, 
                        axis=2).sum(axis=0)
  cond_vs_len_rta.append(cond_rta[2, 2])
  cond_sc = phonons.conductivity(method='sc', max_n_iterations = max_n_iters, 
                        length=length, axis=2)[0].sum(axis=0)
  cond_vs_len_sc.append(cond_sc[2, 2])

print('\n')
plt.figure()
plt.plot(lengths, volume_ratio*np.array(cond_vs_len_rta), '-b', label='rta')
plt.plot(lengths, volume_ratio*np.array(cond_vs_len_sc), '-', label='sc')
plt.legend(loc='best')
plt.xlabel('$\ell$ [nm]',  fontsize=25, fontweight='bold')
plt.ylabel(r"$\kappa$ [W/(m•K)]",  fontsize=25, fontweight='bold')
plt.grid()
plt.xscale('log')
plt.show()